In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from numpy.random import randint
from numpy.random import rand
import numpy as np

pd.options.display.max_columns = 30
%matplotlib inline

### TF-IDF Vector

In [2]:
list = []

In [3]:
#Reading our data
infile = open('train-data_new.dat', 'r')
lines = infile.readlines()
infile.close()

for str in lines:
    list.append(str)
    

In [4]:
new_list = [s.replace("\n", "").strip() for s in list]

In [5]:
new_list[0]

'6705 5997 8310 3606 674 8058 5044 4836  4312 5154 8310 4225'

In [6]:
count_vectorizer = CountVectorizer()

In [7]:
# .fit_transform TOKENIZES and COUNTS
X = count_vectorizer.fit_transform(new_list)

In [8]:
X

<8251x8510 sparse matrix of type '<class 'numpy.int64'>'
	with 710161 stored elements in Compressed Sparse Row format>

In [9]:
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [10]:
pd.DataFrame(X.toarray())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,...,8495,8496,8497,8498,8499,8500,8501,8502,8503,8504,8505,8506,8507,8508,8509
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8246,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8247,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8248,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8249,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
count_vectorizer.get_feature_names()

['10',
 '100',
 '1000',
 '1001',
 '1002',
 '1003',
 '1004',
 '1005',
 '1006',
 '1007',
 '1008',
 '1009',
 '101',
 '1010',
 '1011',
 '1012',
 '1013',
 '1014',
 '1015',
 '1016',
 '1017',
 '1018',
 '1019',
 '102',
 '1020',
 '1021',
 '1022',
 '1023',
 '1024',
 '1025',
 '1026',
 '1027',
 '1028',
 '1029',
 '103',
 '1030',
 '1031',
 '1032',
 '1033',
 '1034',
 '1035',
 '1036',
 '1037',
 '1038',
 '1039',
 '104',
 '1040',
 '1041',
 '1042',
 '1043',
 '1044',
 '1045',
 '1046',
 '1047',
 '1048',
 '1049',
 '105',
 '1050',
 '1051',
 '1052',
 '1053',
 '1054',
 '1055',
 '1056',
 '1057',
 '1058',
 '1059',
 '106',
 '1060',
 '1061',
 '1062',
 '1063',
 '1064',
 '1065',
 '1066',
 '1067',
 '1068',
 '1069',
 '107',
 '1070',
 '1071',
 '1072',
 '1073',
 '1074',
 '1075',
 '1076',
 '1077',
 '1078',
 '1079',
 '108',
 '1080',
 '1081',
 '1082',
 '1083',
 '1084',
 '1085',
 '1086',
 '1087',
 '1088',
 '1089',
 '109',
 '1090',
 '1091',
 '1092',
 '1093',
 '1094',
 '1095',
 '1096',
 '1097',
 '1098',
 '1099',
 '11',
 '110'

In [14]:
pd.DataFrame(X.toarray(), columns=count_vectorizer.get_feature_names())

,10,100,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,101,1010,1011,...,986,987,988,989,99,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8246,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8247,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8248,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8249,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
#tf values
tfidf_vectorizer = TfidfVectorizer(stop_words='english',use_idf=False)
X = tfidf_vectorizer.fit_transform(new_list)
tf = pd.DataFrame(X.toarray(), columns=tfidf_vectorizer.get_feature_names())

In [16]:
#idf values
# use_idf=True is default
idf_vectorizer = TfidfVectorizer(stop_words='english', use_idf=True)
X = idf_vectorizer.fit_transform(new_list)
idf_df = pd.DataFrame(X.toarray(), columns=idf_vectorizer.get_feature_names())
idf_df

,10,100,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,101,1010,1011,...,986,987,988,989,99,990,991,992,993,994,995,996,997,998,999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8246,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
#just a check for specific values
tf['674'], idf_df['674']

(0       0.267261
 1       0.000000
 2       0.064150
 3       0.000000
 4       0.214423
           ...   
 8246    0.000000
 8247    0.000000
 8248    0.091670
 8249    0.000000
 8250    0.000000
 Name: 674, Length: 8251, dtype: float64,
 0       0.100448
 1       0.000000
 2       0.025393
 3       0.000000
 4       0.099453
           ...   
 8246    0.000000
 8247    0.000000
 8248    0.039049
 8249    0.000000
 8250    0.000000
 Name: 674, Length: 8251, dtype: float64)

In [18]:
# Multiply two DataFrames to make the tfidf values

tfidf = tf.mul(idf_df);
print(tfidf);

       10  100  1000  1001  1002  1003  1004  1005  1006  1007  1008  1009  \
0     0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1     0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2     0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3     0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4     0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
...   ...  ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
8246  0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8247  0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8248  0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8249  0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
8250  0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

      101  1010  1011  ...  986  987  988  989   99  990  991  

In [19]:
#check
tfidf["674"]

0       0.026846
1       0.000000
2       0.001629
3       0.000000
4       0.021325
          ...   
8246    0.000000
8247    0.000000
8248    0.003580
8249    0.000000
8250    0.000000
Name: 674, Length: 8251, dtype: float64

In [20]:
#mean tfidf value for every single word
tfidf = tfidf.mean()

In [21]:
tfidf

10      0.000228
100     0.000009
1000    0.000006
1001    0.000063
1002    0.000005
          ...   
995     0.000050
996     0.000009
997     0.000110
998     0.000018
999     0.000403
Length: 8510, dtype: float64

In [22]:
#turning it into DataFrame
tfidf_df = pd.DataFrame([tfidf])

In [23]:
tfidf_df = tfidf_df*100000
tfidf_df

,10,100,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,101,1010,1011,...,986,987,988,989,99,990,991,992,993,994,995,996,997,998,999
0,22.813658,0.906543,0.631449,6.286523,0.48377,3.189167,0.499703,10.882006,39.170193,20.143316,3.314022,2.826585,0.530863,0.595622,1.588791,...,3.39986,1.290644,22.497982,12.974896,0.732116,1.971585,66.865905,1.694545,1.144171,1.553381,4.978499,0.885368,11.030664,1.84274,40.298153


### Initial Population

In [24]:
n_pop = 20

In [25]:
n_bits = 8510

In [26]:
#Creating 4 smaller populations with different probabilities to get various chromosomes
pop1 = [np.random.choice(np.arange(0, 2), p=[0.8, 0.2], size=n_bits).tolist() for _ in range(int(n_pop/4))]
pop2 = [np.random.choice(np.arange(0, 2), p=[0.6, 0.4], size=n_bits).tolist() for _ in range(int(n_pop/4))]
pop3 = [np.random.choice(np.arange(0, 2), p=[0.9, 0.1], size=n_bits).tolist() for _ in range(int(n_pop/4))]
pop4 = [np.random.choice(np.arange(0, 2), p=[0.7, 0.3], size=n_bits).tolist() for _ in range(int(n_pop/4))]

In [27]:
pop = pop1 + pop2 + pop3 + pop4

In [28]:
pop

[[1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,


In [29]:
pop_df = pd.DataFrame(pop, columns=idf_vectorizer.get_feature_names())

In [30]:
pop_df

,10,100,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,101,1010,1011,...,986,987,988,989,99,990,991,992,993,994,995,996,997,998,999
0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0
2,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0
4,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0
5,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,...,1,1,0,0,0,0,0,1,0,1,0,1,0,1,0
6,1,1,1,1,0,0,1,0,1,0,1,0,1,0,1,...,1,1,1,1,1,0,1,1,0,0,0,0,0,0,0
7,1,0,1,0,0,0,0,1,0,0,1,1,1,1,1,...,0,0,1,1,0,1,1,0,0,0,0,0,1,0,0
8,0,1,0,0,0,0,0,1,1,1,0,1,0,0,0,...,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0
9,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,1,0,0,1,0,1,0,1,0,1


In [31]:
#Changing the value of every selected word from 1 to its tfidf value
tfidf_values = pd.DataFrame(columns=idf_vectorizer.get_feature_names());
    
for i in range(len(pop_df)) :
    tfidf_values = tfidf_values.append(pop_df.iloc[i,:]*tfidf_df)

In [35]:
tfidf_values

,10,100,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,101,1010,1011,...,986,987,988,989,99,990,991,992,993,994,995,996,997,998,999
0,22.813658,0.000000,0.631449,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.826585,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,66.865905,0.000000,1.144171,0.000000,0.000000,0.000000,11.030664,0.00000,0.000000
0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.595622,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.732116,0.000000,66.865905,0.000000,0.000000,0.000000,0.000000,0.000000,11.030664,0.00000,0.000000
0,0.000000,0.906543,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.826585,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
0,0.000000,0.000000,0.000000,0.000000,0.00000,3.189167,0.000000,10.882006,0.000000,0.000000,0.000000,2.826585,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,66.865905,1.694545,0.000000,0.000000,0.000000,0.000000,11.030664,0.00000,0.000000
0,0.000000,0.000000,0.000000,6.286523,0.00000,0.000000,0.000000,10.882006,0.000000,0.000000,0.000000,2.826585,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.694545,0.000000,0.000000,0.000000,0.885368,11.030664,0.00000,0.000000
0,0.000000,0.000000,0.000000,0.000000,0.00000,3.189167,0.000000,0.000000,0.000000,20.143316,3.314022,0.000000,0.000000,0.000000,0.000000,...,3.39986,1.290644,0.000000,0.000000,0.000000,0.000000,0.000000,1.694545,0.000000,1.553381,0.000000,0.885368,0.000000,1.84274,0.000000
0,22.813658,0.906543,0.631449,6.286523,0.00000,0.000000,0.499703,0.000000,39.170193,0.000000,3.314022,0.000000,0.530863,0.000000,1.588791,...,3.39986,1.290644,22.497982,12.974896,0.732116,0.000000,66.865905,1.694545,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
0,22.813658,0.000000,0.631449,0.000000,0.00000,0.000000,0.000000,10.882006,0.000000,0.000000,3.314022,2.826585,0.530863,0.595622,1.588791,...,0.00000,0.000000,22.497982,12.974896,0.000000,1.971585,66.865905,0.000000,0.000000,0.000000,0.000000,0.000000,11.030664,0.00000,0.000000
0,0.000000,0.906543,0.000000,0.000000,0.00000,0.000000,0.000000,10.882006,39.170193,20.143316,0.000000,2.826585,0.000000,0.000000,0.000000,...,0.00000,1.290644,0.000000,0.000000,0.000000,1.971585,0.000000,1.694545,0.000000,0.000000,0.000000,0.000000,11.030664,1.84274,0.000000
0,0.000000,0.000000,0.000000,0.000000,0.48377,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.826585,0.000000,0.000000,0.000000,...,0.00000,1.290644,0.000000,0.000000,0.000000,1.971585,0.000000,0.000000,1.144171,0.000000,4.978499,0.000000,11.030664,0.00000,40.298153


In [36]:
#Mean tfidf value of all selected words for every chromosome
tfidf_mean = tfidf_values.sum(axis = 1)
tfidf_mean = tfidf_mean/(tfidf_values > 0).sum(axis = 1)
tfidf_mean

In [39]:
selected_words = (tfidf_values > 0).sum(axis = 1)
selected_words

#### Penalty Function

In [41]:
def penalty(x):
        return (abs(x-1000)/100)

In [42]:
for s in selected_words:
    print(format(penalty(s), ".3f"))

6.720
6.700
7.000
7.440
6.650
23.700
24.990
24.340
24.490
23.840
0.980
1.450
1.950
1.290
1.750
15.380
15.800
14.570
15.760
15.880


In [53]:
s1 = pd.Series(selected_words, name='selected_words')

s2 = pd.Series(tfidf_mean, name='tfidf')

final = pd.concat([s1, s2], axis=1)
final = final.reset_index()
final = final.drop('index', 1)
final

<ipython-input-53-36d8bb2ed0bf>:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  final = final.drop('index', 1)


,selected_words,tfidf
0,1672,10.855992
1,1670,10.684483
2,1700,11.528627
3,1744,10.989911
4,1665,10.908105
5,3370,11.052928
6,3499,10.781246
7,3434,11.168269
8,3449,11.056659
9,3384,10.593673


In [78]:
final.iloc[:,1].values # first column of data frame (first_name)

array([10.85599177, 10.68448317, 11.52862743, 10.9899109 , 10.9081049 ,
       11.05292761, 10.78124596, 11.16826892, 11.05665941, 10.59367304,
        9.98551   , 11.4257645 , 10.86095124, 12.78876409, 10.77527916,
       11.1668104 , 10.97687054, 11.96767118, 10.8755936 , 11.10655132])

#### Fitness Function

In [54]:
def fitness(df):
    x = []
    for i in range(len(df)) :
        if df.iloc[i,0] < 1000:
            x.append([0])
        else:
            x.append([df.iloc[i,1]**2 - penalty(df.iloc[i,0])])
    return pd.DataFrame(x, columns = ['fit'])

In [55]:
fitness(final)

,fit
0,111.132557
1,107.458181
2,125.909250
3,113.338142
4,112.336753
5,98.467209
6,91.245264
7,100.390231
8,97.759717
9,88.385908


### Selection Function / Roulette Wheel

In [125]:
def roulette_wheel_selection(population):

    # Computes the totallity of the population fitness
    population_fitness = fitness(population).sum()
    
    chromosome_probabilities = pd.DataFrame()
    x = fitness(population)
    
    # Computes for each chromosome the probability 
    for row in x.itertuples():
        chromosome_probabilities = chromosome_probabilities.append(row.fit/population_fitness, ignore_index=True)
    
    chromosome_probabilities.rename(columns = {'fit':'Prob'}, inplace = True)
    #np.random.choice(np.arange(0, len(population)), p=[chromosome_probabilities.iloc[:,0].values])
    # Returns the probabilities for every chromosome in the roulette
    return chromosome_probabilities

In [124]:
#np.random.choice(np.arange(0, len(final)), p=[x.iloc[:,0].values])
#x = roulette_wheel_selection(final)
#np.array(x.iloc[:,0].values)

In [126]:
roulette_wheel_selection(final).sum()

Prob    1.0
dtype: float64

### Crossover Function / Single Point

In [189]:
# crossover two parents to create two children
def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if rand() < r_cross:
        pt = randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

### Mutation Function

In [190]:
# mutation operator
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        # check for a mutation
        if rand() < r_mut:
            # flip the bit
            bitstring[i] = 1 - bitstring[i]


### GA IMPLEMENTATION-NOT FINISHED

In [209]:
# genetic algorithm
def genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross, r_mut):
    # initial population of random bitstring
    pop1 = [np.random.choice(np.arange(0, 2), p=[0.8, 0.2], size=n_bits).tolist() for _ in range(int(n_pop/4))]
    pop2 = [np.random.choice(np.arange(0, 2), p=[0.6, 0.4], size=n_bits).tolist() for _ in range(int(n_pop/4))]
    pop3 = [np.random.choice(np.arange(0, 2), p=[0.9, 0.1], size=n_bits).tolist() for _ in range(int(n_pop/4))]
    pop4 = [np.random.choice(np.arange(0, 2), p=[0.7, 0.3], size=n_bits).tolist() for _ in range(int(n_pop/4))]
    pop = pop1 + pop2 + pop3 + pop4
    pop_df = pd.DataFrame(pop, columns=idf_vectorizer.get_feature_names())
    
    # keep track of best solution
    best, best_eval = 0, objective(pop_df[0])
    # enumerate generations
    for gen in range(n_iter):
        # evaluate all candidates in the population
        scores = [objective(c) for c in pop_df]
        # check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop_df[i], scores[i]
                print(">%d, new best f(%s) = %.3f" % (gen,  pop_df[i], scores[i]))
        # select parents
        selected = [roulette_wheel_selection(pop_df, scores) for _ in range(n_pop)]
        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace population
        pop_df = children
    return [best, best_eval]


In [210]:
# define the total iterations
n_iter = 100
# bits
n_bits = 8510
# define the population size
n_pop = 20
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / float(n_bits)
# perform the genetic algorithm search
best, score = genetic_algorithm(penalty, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
print('f(%s) = %f' % (best, score))

KeyError: 0